<a href="https://colab.research.google.com/github/falon-go-weeee/Colab-Notebooks/blob/main/SH_falcon7b_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Install required Libraries

In [ ]:
!pip install transformers accelerate einops xformers fastapi[all] uvicorn[standard]

In [ ]:
from subprocess import getoutput
from IPython.display import clear_output
import os
import time

Tunnel_type = "bore" #@param ["ngrok", "bore", "cloudflare"]
binding_port = "8000" #@param {type:"string"}
port = "34525" #@param {type:"string"}

def install_ngrok():
  if not os.path.exists('/usr/local/lib/python3.8/dist-packages/pyngrok'):
    !pip install pyngrok
    clear_output()

def install_bore():
  if not os.path.exists('/usr/bin/bore'):
    !wget https://github.com/ekzhang/bore/releases/download/v0.4.1/bore-v0.4.1-i686-unknown-linux-musl.tar.gz
    !tar -xf bore-v0.4.1-i686-unknown-linux-musl.tar.gz
    !rm -f bore-v0.4.1-i686-unknown-linux-musl.tar.gz
    !cp bore /usr/bin/bore
    !rm -rf bore
    clear_output()

def install_cloudflare():
  if not os.path.exists('/usr/bin/cloudflared'):
    !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb && dpkg -i cloudflared-linux-amd64.deb
    !rm -f cloudflared-linux-amd64.deb
    clear_output()

if Tunnel_type == "bore":
  install_bore()
  use_ip = ""
  if port:
    # !curl ifconfig.me > ip.txt
    # clear_output()
    # ip = getoutput('cat /content/ip.txt')
    # ip = f"-l {ip}"
    !nohup bore local $binding_port --to bore.pub -p $port > /content/tunnel_srv.txt 2>&1 &
  else:
    !nohup bore local $binding_port --to bore.pub > /content/tunnel_srv.txt 2>&1 &
  time.sleep(3)
  !grep -o 'bore.pub:[^ ]*' /content/tunnel_srv.txt >/content/tunnel_srvr.txt
  time.sleep(3)
  tunnel_srv= getoutput('cat /content/tunnel_srvr.txt')
  if tunnel_srv == '':
    print('port not available choose another port')
  else:
    print('http://'+tunnel_srv)
  !rm /content/tunnel_srv.txt
  !rm /content/tunnel_srvr.txt

elif Tunnel_type == "ngrok":
  install_ngrok()
  from pyngrok import ngrok
  # Terminate ngrok port
  ngrok.kill()
  # Set authentication (optional)
  # Get your authentication token via https://dashboard.ngrok.com/auth
  NGROK_AUTH_TOKEN = "2H79LWzHBonanC1xWxoGXW6fvtY_5rD7uShKgACuSLvUwMvJA"
  ngrok.set_auth_token(NGROK_AUTH_TOKEN)
  public_url = ngrok.connect(binding_port)
  print(public_url)

elif Tunnel_type == "cloudflare":
  install_cloudflare()
  !pkill cloudflared
  time.sleep(4)
  # Tunnel_link = "& cloudflared tunnel --url http://localhost:7860"
  !nohup cloudflared tunnel --url http://localhost:$binding_port > /content/srv.txt 2>&1 &
  time.sleep(4)
  !grep -o 'https[^[:space:]]*\.trycloudflare.com' /content/srv.txt >/content/srvr.txt
  srv= getoutput('cat /content/srvr.txt')
  print(srv)
  time.sleep(4)
  !rm /content/srv.txt /content/srvr.txt

http://bore.pub:34525


In [ ]:
!python -u "/content/drive/MyDrive/Colab Notebooks/falcon-7b-api.py"

2023-06-11 07:30:40.823829: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading checkpoint shards: 100% 2/2 [01:18<00:00, 39.47s/it]
The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 

In [ ]:
#@markdown ## kill process

import os
import signal
import psutil

process_name = "bore" #@param {type:"string"}

for proc in psutil.process_iter(['pid', 'name']):
  if proc.info['name'] == process_name:
      os.kill(proc.pid, signal.SIGTERM)
!ps -aux | grep $process_name